In [1]:
from datetime import datetime
import datetime as dt
import os
from operator import itemgetter
import shutil 
import pickle
import time
import json

from django.contrib.gis.geos.geometry import GEOSGeometry
from osgeo import osr
from pyproj import Proj
from fiona.crs import from_epsg
from fiona.crs import to_string
from sklearn.cluster import KMeans
import geopandas as gpd
from affine import Affine
import xarray as xr
import numpy as np
from rasterio.features import rasterize
from shapely.geometry import shape
import fiona
import glob
from dask.distributed import Client
from datacube.drivers.netcdf import write_dataset_to_netcdf
from datacube.api import GridWorkflow

from madmex.wrappers import gwf_query
from madmex.util.xarray import to_float, to_int
from madmex.models import PredictObject, TrainClassification
from madmex.models import Region

2020-02-21 00:52:25,726 - datacube.drivers.driver_cache - driver_cache safe_load: Failed to resolve driver datacube.plugins.index::s3aio_index
2020-02-21 00:52:25,728 - datacube.drivers.driver_cache - driver_cache safe_load: Error was: UndefinedEnvironmentName("'extra' does not exist in evaluation environment.",)
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:32774 remote=tcp://127.0.0.1:42283>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:32776 remote=tcp://127.0.0.1:42283>
distributed.comm.tcp - WARNING - Closing dangling stream in <TCP  local=tcp://127.0.0.1:32778 remote=tcp://127.0.0.1:42283>
/home/madmex_user/.local/lib/python3.6/site-packages/distributed/worker.py:3269: UserWarning: Large object of size 25.02 MB detected in task graph: 
2020-02-21 01:07:33,162 - madmex.wrappers - loggerwriter write: /home/madmex_user/.local/lib/python3.6/site-packages/distributed/worker.py:3269: UserWarning: Large 

In [2]:
client = Client(n_workers=5,memory_limit='110GB', threads_per_worker=1)

In [3]:
client

Client Scheduler: tcp://127.0.0.1:42283 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 550.00 GB


In [4]:
client.restart()

Client Scheduler: tcp://127.0.0.1:42283 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 550.00 GB


In [5]:
region = 'cultivos_2019_simplify'
name_of_product = 's2_l2a_10m_scl_mexico'
begin = '2018-01-01'
end = '2019-12-31'
gwf_kwargs = {'region': region, 'product': name_of_product, 'begin': begin, 'end':end}
iterable = gwf_query(**gwf_kwargs)

In [6]:
lista_iter = list(iterable)

In [7]:
len(lista_iter)

53

In [11]:
[elem[0] for elem in lista_iter]

[(47, -35),
 (47, -37),
 (48, -37),
 (47, -36),
 (48, -36),
 (40, -31),
 (39, -31),
 (38, -30),
 (39, -30),
 (40, -30),
 (39, -29),
 (38, -29),
 (40, -29),
 (38, -28),
 (40, -28),
 (38, -27),
 (39, -28),
 (39, -27),
 (38, -26),
 (39, -26),
 (40, -33),
 (40, -32),
 (39, -32),
 (37, -29),
 (40, -34),
 (41, -33),
 (42, -33),
 (41, -32),
 (42, -32),
 (41, -31),
 (42, -31),
 (43, -33),
 (44, -34),
 (43, -35),
 (45, -34),
 (44, -35),
 (45, -37),
 (43, -34),
 (44, -37),
 (43, -36),
 (45, -36),
 (45, -35),
 (44, -36),
 (42, -34),
 (41, -34),
 (37, -27),
 (37, -26),
 (37, -28),
 (41, -30),
 (46, -35),
 (46, -34),
 (46, -36),
 (46, -37)]

# 1) Se elige el primer dc_tile de lista_iter para la generación de segmentos clusterizados

In [12]:
date_today = datetime.today().strftime("%m-%d-%Y")

In [13]:
index=49

In [17]:
var=lista_iter[index]

In [18]:
dc_tile = '%d_%d' % (var[0][0], var[0][1])

In [19]:
dc_tile

'46_-35'

In [20]:
sr_0 = GridWorkflow.load(var[1],dask_chunks={'x': 1600, 'y': 1600,'time': 60})

In [21]:
sr_0.attrs['geobox'] = var[1].geobox

In [22]:
sr_0 = client.persist(sr_0)

**110gb is enough**

In [23]:
sr_0 = sr_0.apply(func=to_float, keep_attrs=True)

In [24]:
sr_1 = sr_0.where(sr_0.pixel_qa.isin([2,4,5,6,7,8,11]))

In [25]:
sr_1['ndvi'] = ((sr_1.nir - sr_1.red) / (sr_1.nir + sr_1.red)) * 10000

In [26]:
sr_1['ndvi'].attrs['nodata'] = 0

In [27]:
sr_1['gndvi'] = ((sr_1.nir - sr_1.green) / (sr_1.nir + sr_1.green)) * 10000

In [28]:
sr_1['gndvi'].attrs['nodata'] = 0

In [29]:
sr_1['avi'] = ((sr_1.nir*(1-sr_1.red))*(sr_1.nir-sr_1.red))**(1/3)

In [30]:
sr_1['avi'].attrs['nodata'] = 0

In [31]:
sr_1 = client.persist(sr_1)

# 2) Cálculo de features

In [32]:
ndvi_mean_resampled = sr_1.ndvi.resample(time='M').mean()

In [33]:
ndvi_mean_resampled = client.persist(ndvi_mean_resampled)

In [34]:
ndvi_mean_resampled_drop_na = ndvi_mean_resampled.dropna('time',how='all')

In [35]:
gndvi_mean_resampled = sr_1.gndvi.resample(time='M').mean()

In [36]:
gndvi_mean_resampled = client.persist(gndvi_mean_resampled)

In [37]:
gndvi_mean_resampled_drop_na = gndvi_mean_resampled.dropna('time',how='all')

In [38]:
avi_mean_resampled = sr_1.avi.resample(time='M').mean()

In [39]:
avi_mean_resampled = client.persist(avi_mean_resampled)

In [40]:
avi_mean_resampled_drop_na = avi_mean_resampled.dropna('time',how='all')

In [41]:
combined = xr.merge([ndvi_mean_resampled_drop_na,
                     gndvi_mean_resampled_drop_na,
                     avi_mean_resampled_drop_na])

In [42]:
combined = client.persist(combined)

In [43]:
features = list(combined.data_vars)
s = '-'
features_string = s.join(features)

In [44]:
path_result = '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/' + features_string + '/'

In [45]:
path_result_dc_tile = os.path.join(path_result, date_today 
                                   + '/' + dc_tile + '/')

In [46]:
path_result_dc_tile

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-21-2020/46_-35/'

In [47]:
if not os.path.exists(path_result_dc_tile):
    os.makedirs(path_result_dc_tile)

In [48]:
nc_filename = path_result_dc_tile + date_today + '_combined_' + dc_tile + '.nc'

In [49]:
combined.attrs = sr_0.attrs

In [50]:
combined.coords['time'].attrs = sr_0.coords['time'].attrs
combined.coords['x'].attrs = sr_0.coords['x'].attrs
combined.coords['y'].attrs = sr_0.coords['y'].attrs

In [51]:
#write_dataset_to_netcdf(combined.compute(),nc_filename)

# 3) Leer segmentación

In [52]:
geom = GEOSGeometry(json.dumps(sr_0.geobox.geographic_extent.json))

In [53]:
seg_name = 'seg_mex_s2_10m_scl_2018_2019'

In [54]:
qs = PredictObject.objects.filter(the_geom__contained=geom,segmentation_information__name=seg_name)

In [55]:
path_seg = qs[0].path

In [57]:
with fiona.open(path_seg) as src:
    fc = list(src)
    crs = to_string(src.crs)

In [58]:
fc_subset=((x['properties']['id'], x['geometry']) for x in fc)

In [59]:
fc_sorted = sorted(fc_subset, key=itemgetter(0))

**First position of fc_sorted is id, second position is geometry**


In [60]:
iterable_zip = zip([k[1] for k in fc_sorted], [k[0] for k in fc_sorted])

In [61]:
aff = Affine(*list(sr_0.affine)[0:6])

In [62]:
dimensions_dataset = list(sr_0.coords)
list_dimensions = [x for x in dimensions_dataset if x != 'time']
lambda_function = lambda l_netcdf,l_test: l_netcdf[0] if l_netcdf[0] in l_test else l_netcdf[1]
xdim = lambda_function(list_dimensions,['x','longitude'])
ydim = lambda_function(list_dimensions,['y','latitude'])

In [63]:
fc_raster = rasterize(iterable_zip, transform=aff,
                      out_shape=(sr_0.sizes[ydim], sr_0.sizes[xdim]),
                      dtype='float64', fill=np.nan)

In [64]:
fc_raster = fc_raster.astype(int)

In [65]:
fc_dataarray = xr.DataArray(fc_raster, dims=[ydim, xdim], name='features_id')

# 4) Construcción de dataframes para pasar al kmeans

In [66]:
combined = xr.merge([fc_dataarray,combined])

In [67]:
combined = client.persist(combined)

In [68]:
combined = combined.stack(z=(xdim, ydim))
combined = combined.reset_index('z')
combined = combined.drop([xdim,ydim])


In [69]:
combined = combined.where((np.isfinite(combined['features_id'])), drop=True)

In [70]:
combined = client.persist(combined)

In [71]:
df = combined.to_dataframe()

In [72]:
del sr_0
del ndvi_mean_resampled_drop_na
del gndvi_mean_resampled_drop_na
del avi_mean_resampled_drop_na

In [73]:
client.restart()

Client Scheduler: tcp://127.0.0.1:42283 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 5 Memory: 550.00 GB


In [74]:
df=df.reset_index('time')

In [75]:
df2 = df.groupby(['time','features_id'],as_index=False).agg({features[0]: 'median',
                                                             features[1]: 'median',
                                                             features[2]: 'median'})

In [76]:
df2.index = df2['time']

In [77]:
df2.drop(['time'], axis=1, inplace=True)

In [78]:
df_filename = path_result_dc_tile + date_today + '_df' + '.csv'

In [79]:
df2.to_csv(df_filename)

In [80]:
times_df = [datetime.date(d).strftime('%Y-%m-%d') for d in df2.index.get_level_values('time').unique()]

In [81]:
df3 = df2.loc[times_df[0]]

In [82]:
df3.rename(columns={features[0]: features[0]+'_' + times_df[0],
                    features[1]: features[1]+'_' +times_df[0],
                    features[2]: features[2]+'_' +times_df[0]},
                 inplace=True)

In [83]:
df3=df3.reset_index('time')

In [84]:
df3.drop(['time'],axis=1,inplace=True)

In [85]:
for k in range(1,len(times_df)):
    df_subset = df2.loc[times_df[k]]
    df_subset.rename(columns={features[0]: features[0]+'_' + times_df[k],
                        features[1]: features[1]+'_' +times_df[k],
                        features[2]: features[2]+'_' +times_df[k]},
                     inplace=True)
    df_subset=df_subset.reset_index('time')
    df_subset.drop(['time'],axis=1,inplace=True)
    df3 = df3.join(df_subset.set_index('features_id'), on='features_id')

**Join df with fc in geopandas dataframe form**

In [86]:
crs_dict = {'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [87]:
gdf = gpd.GeoDataFrame.from_features([feature for feature in fc], 
                                      crs = crs_dict)


In [88]:
gdf = gdf.rename(columns={'id':'features_id'})

In [89]:
gdf_df_join = gdf.merge(df3, on='features_id', how='left')

In [90]:
#obj_region = Region.objects.filter(name='mexico_Veracruz')[0]

In [91]:
#obj_region

In [92]:
#geojson_region_lcc=json.loads(obj_region.the_geom.transform(to_string(crs_dict),clone=True).geojson)

In [93]:
poly = qs[0].the_geom.transform(to_string(crs_dict),clone=True)

In [94]:
poly_geojson = poly.geojson

In [95]:
geometry_seg = json.loads(poly_geojson)

In [367]:
#shape_region_cut = shape(geojson_region_lcc).intersection(shape(geometry_seg))

In [369]:
#gs_aoi = gpd.GeoSeries([shape_region_cut]) 

In [370]:
#gd_aoi = gpd.GeoDataFrame()

In [371]:
#gd_aoi['geometry'] = gs_aoi

In [96]:
shape_region = shape(geometry_seg)

In [97]:
gs_region = gpd.GeoSeries([shape_region]) 

In [98]:
gd_region = gpd.GeoDataFrame()

In [99]:
gd_region['geometry'] = gs_region

In [135]:
#input_file = '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/madmex_masks_agricultura_pastizal/Veracruz_de_Ignacio_de_la_Llave_sentinel2_2018_agricultura_pastizal_lcc_no_inegi.shp'
#input_file = '/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/madmex_masks_agricultura_pastizal/Veracruz_de_Ignacio_de_la_Llave_sentinel2_2018_agricultura_pastizal_lcc_simplified.shp'
input_file='/shared_volume/tasks/2020/Veracruz_de_Ignacio_de_la_Llave_sentinel2_2018_agricultura_pastizal_simplified_centroids_no_inegi.shp'

In [136]:
gdf_df_mask = gpd.read_file(input_file)

In [137]:
gdf_df_mask.head()

,descr_31,clave_31,geometry
0,Pastizales,27,POINT (2945406.963 1060784.660)
1,Pastizales,27,POINT (2944995.058 1060774.620)
2,Pastizales,27,POINT (2945304.602 1060756.115)
3,Tierras Agrcolas,28,POINT (2938941.618 1060581.627)
4,Tierras Agrcolas,28,POINT (2941259.331 1060423.758)


In [138]:
len(gdf_df_mask.index)

194196

In [139]:
gdf_df_mask.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'ellps': 'WGS84',
 'units': 'm',
 'no_defs': True}

In [140]:
crs_dict

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [141]:
gd_region.crs = crs_dict

In [142]:
gd_region.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [143]:
gd_df_mask_intersects = gpd.sjoin(gdf_df_mask.reset_index(drop=True),
                                       gd_region.reset_index(drop=True),
                                       op='intersects',
                                       how='inner')

In [144]:
gd_df_mask_intersects.head()

,descr_31,clave_31,geometry,index_right
7478,Tierras Agrcolas,28,POINT (3328338.029 713362.988),0
7479,Tierras Agrcolas,28,POINT (3328276.779 713329.238),0
7482,Pastizales,27,POINT (3328155.529 713285.488),0
7489,Pastizales,27,POINT (3326804.098 712908.418),0
7493,Pastizales,27,POINT (3323716.779 711939.238),0


In [145]:
gd_df_mask_intersects.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'ellps': 'WGS84',
 'units': 'm',
 'no_defs': True}

In [146]:
gd_df_mask_intersects_reduced = gpd.GeoDataFrame()
gd_df_mask_intersects_reduced['geometry'] = gd_df_mask_intersects['geometry']

In [147]:
gd_df_mask_intersects_reduced.head()

,geometry
7478,POINT (3328338.029 713362.988)
7479,POINT (3328276.779 713329.238)
7482,POINT (3328155.529 713285.488)
7489,POINT (3326804.098 712908.418)
7493,POINT (3323716.779 711939.238)


In [148]:
gd_df_mask_intersects_reduced.crs =crs_dict

In [149]:
gd_df_mask_intersects_reduced.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [150]:
gdf_df_join.crs

{'proj': 'lcc',
 'lat_1': 17.5,
 'lat_2': 29.5,
 'lat_0': 12,
 'lon_0': -102,
 'x_0': 2500000,
 'y_0': 0,
 'datum': 'WGS84',
 'units': 'm',
 'no_defs': True,
 'wktext': True}

In [151]:
start=time.time()
gdf_df_join_intersects_aoi = gpd.sjoin(gdf_df_join.reset_index(drop=True),
                                       gd_df_mask_intersects_reduced.reset_index(drop=True),
                                       op='intersects',
                                       how='inner')
end=time.time()

In [152]:
gdf_df_join_intersects_aoi.head()

,geometry,features_id,ndvi_2018-10-31,gndvi_2018-10-31,avi_2018-10-31,ndvi_2018-11-30,gndvi_2018-11-30,avi_2018-11-30,ndvi_2018-12-31,gndvi_2018-12-31,...,ndvi_2019-07-31,gndvi_2019-07-31,avi_2019-07-31,ndvi_2019-08-31,gndvi_2019-08-31,avi_2019-08-31,ndvi_2019-09-30,gndvi_2019-09-30,avi_2019-09-30,index_right
29924,"POLYGON ((3278350.000 715810.000, 3278390.000 ...",435202,-5343.465046,-4526.472782,735.021453,-5902.318073,-6009.640942,250.17719,-6830.082551,-7307.598011,...,-2180.990067,-3047.689208,515.903631,-5419.842679,-6050.984256,367.528775,-6539.955808,-7212.743231,238.083178,1028
31264,"POLYGON ((3278640.000 715460.000, 3278690.000 ...",433974,5173.420957,4711.504425,NaN,6740.743473,5524.455125,NaN,7403.520350,6112.699619,...,4538.403536,3083.881656,287.834363,6032.378806,4608.251345,NaN,7058.195450,5444.422611,NaN,1590
32504,"POLYGON ((3278790.000 715230.000, 3278850.000 ...",432482,7900.790798,6542.421015,NaN,7530.794432,6300.056835,NaN,8304.234673,7115.724182,...,7037.948950,5811.627514,NaN,7184.438778,5958.051745,NaN,7442.472094,6167.926998,NaN,1591
33351,"POLYGON ((3278760.000 715150.000, 3278770.000 ...",431883,7991.420291,6617.969903,NaN,7936.880321,6648.898335,NaN,8309.514333,7206.005750,...,7273.692765,6079.627441,NaN,7664.220224,6443.196755,NaN,7886.272934,6524.682504,NaN,1592
43234,"POLYGON ((3279330.000 713260.000, 3279360.000 ...",422020,8602.693603,7224.113759,NaN,7960.347158,6719.512241,NaN,7759.775871,6949.855367,...,6853.836109,6097.494336,NaN,7082.364633,6265.005535,NaN,7791.407084,6648.518376,NaN,1593


In [153]:
len(gdf_df_join_intersects_aoi.index)

3063

# 5) Unsupervised clustering with Kmeans

**Número de clústers depende del número de cultivos presentes en el dc_tile** 

**Revisar credenciales de db**

In [154]:
var[1].geobox.geographic_extent.json

{'type': 'Polygon',
 'coordinates': [[(-94.62445901386619, 18.307292421720017),
   (-94.64874005892365, 17.85809774251316),
   (-94.1770621971074, 17.834324723281572),
   (-94.15122923779683, 18.283423396251024),
   (-94.62445901386619, 18.307292421720017)]]}

In [155]:
training_set = 'cultivos2019_inecol'

In [156]:
qs = TrainClassification.objects.filter(train_object__the_geom__contained=geom,
                                        training_set=training_set).prefetch_related('interpret_tag')

In [157]:
l_tag_ids = []

In [158]:
for element in qs:
    l_tag_ids.append(element.interpret_tag.id)

In [159]:
nclusters = len(np.unique(l_tag_ids))

In [161]:
kmeans = KMeans(n_clusters=nclusters, n_jobs=10)

In [165]:
features_for_kmeans = [column for column in list(gdf_df_join_intersects_aoi.columns) if column not in ['geometry','features_id','index_right']]

In [167]:
kmeans.fit(gdf_df_join_intersects_aoi[features_for_kmeans].fillna(0).values)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=7, n_init=10, n_jobs=10, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

**Write model fit**

In [168]:
model_filename = path_result_dc_tile + date_today + '_model_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi' + '.pkl'

In [169]:
model_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-21-2020/46_-35/02-21-2020_model_46_-35_nclusters_7_aoi.pkl'

In [170]:
pickle.dump(kmeans, open(model_filename, "wb"))

**Predict with model**

In [171]:
model_filename = path_result_dc_tile + date_today + '_model_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi' + '.pkl'

In [172]:
model_filename

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-21-2020/46_-35/02-21-2020_model_46_-35_nclusters_7_aoi.pkl'

In [173]:
kmeans = pickle.load(open(model_filename, "rb"))

**Predict**

In [175]:
preds = kmeans.predict(gdf_df_join_intersects_aoi[features_for_kmeans].fillna(0).values)

In [179]:
gdf_df_clusters = gdf_df_join_intersects_aoi.drop(features_for_kmeans,axis=1)
gdf_df_clusters = gdf_df_clusters.drop('index_right',axis=1)

In [180]:
gdf_df_clusters['preds'] = preds

In [181]:
gdf_df_clusters.head()

,geometry,features_id,preds
29924,"POLYGON ((3278350.000 715810.000, 3278390.000 ...",435202,5
31264,"POLYGON ((3278640.000 715460.000, 3278690.000 ...",433974,0
32504,"POLYGON ((3278790.000 715230.000, 3278850.000 ...",432482,4
33351,"POLYGON ((3278760.000 715150.000, 3278770.000 ...",431883,4
43234,"POLYGON ((3279330.000 713260.000, 3279360.000 ...",422020,4


# 6) Write results to file

**Geopackage**

In [182]:
path_results_geopackage = '/home/madmex_user/agriculture_geopackages/features_computed/' + features_string + '/'

In [183]:
path_result_geopackage = os.path.join(path_results_geopackage, date_today 
                                       + '/' + dc_tile + '/')

In [184]:
if not os.path.exists(path_result_geopackage):
    os.makedirs(path_result_geopackage)

In [185]:
clusters_filename = path_result_geopackage + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi' + '.gpkg'

In [186]:
layer_name = date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi'

In [187]:
gdf_df_clusters.crs = crs_dict

In [188]:
gdf_df_clusters.to_file(clusters_filename,
                        layer=layer_name,
                        driver='GPKG')

In [189]:
shutil.move(clusters_filename,path_result_dc_tile)

'/LUSTRE/MADMEX/tasks/2020/1_clusterization_for_agriculture_labeling/features_computed/ndvi-gndvi-avi/02-21-2020/46_-35/02-21-2020_clusters_46_-35_nclusters_7_aoi.gpkg'

**Shapefile**

In [190]:
clusters_filename = path_result_dc_tile + date_today + '_clusters_' +  dc_tile + '_nclusters_%d' % nclusters + '_aoi' + '.shp'

In [191]:
gdf_df_clusters.to_file(clusters_filename)
